In [ ]:
from flask import render_template_string, Flask, request, jsonify, send_file, redirect
from transformers import AutoProcessor, AutoModelForCausalLM
from datumaro import Bbox, Dataset, DatasetItem
from typing import Dict, Any, Optional, Tuple
from datetime import datetime
from pathlib import Path
from PIL import Image

import supervision as sv
import numpy as np
import logging
import sqlite3
import base64
import torch
import json
import sys
import io
import os

# Configuração de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('oil_detection_server.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

HOME = os.getcwd() # Configuração de caminhos para GroundingDINO
CONFIG_PATH = os.path.join(HOME, "GroundingDINO", "groundingdino", "config", "GroundingDINO_SwinT_OGC.py")

In [ ]:
%cd {HOME}/GroundingDINO
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "GroundingDINO\\weights", WEIGHTS_NAME)
from groundingdino.util.inference import load_model, load_image, predict, annotate
%cd {HOME}

In [27]:
class DatabaseManager:
    """Gerenciador do banco de dados SQLite com estrutura normalizada por empresa"""
    
    def __init__(self, base_path: str = "Overseas"):
        self.base_path = Path(base_path)
        self.base_path.mkdir(exist_ok=True)
        self._databases = {}  # Cache de conexões por empresa
    
    def get_db_path(self, empresa_id: str) -> Path:
        """Retorna o caminho do banco de dados específico da empresa"""
        empresa_dir = self.base_path / empresa_id
        empresa_dir.mkdir(exist_ok=True)
        return empresa_dir / f"{empresa_id}_detection.db"
    
    def init_database(self, empresa_id: str):
        """Inicializa o banco de dados específico da empresa com as tabelas normalizadas"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        # Tabela de empresa (uma única empresa por banco)
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS empresa (
                empresa_id TEXT PRIMARY KEY,
                empresa_nome TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Tabela de coletores da empresa
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS coletores (
                coletor_id TEXT PRIMARY KEY,
                coletor_descricao TEXT,
                coletor_localizacao TEXT,
                status TEXT DEFAULT 'ativo',
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Tabela principal de detecções
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS deteccoes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                coletor_id TEXT NOT NULL,
                timestamp_coleta TIMESTAMP NOT NULL,
                timestamp_processamento TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                modelo_usado TEXT NOT NULL,
                imagem_original_path TEXT NOT NULL,
                imagem_processada_path TEXT,
                confidence_yolo REAL,
                objects_detected_yolo INTEGER DEFAULT 0,
                FOREIGN KEY (coletor_id) REFERENCES coletores (coletor_id)
            )
        ''')
        
        # Tabela para captions do Florence-2 (normalizada)
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS florence_captions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                deteccao_id INTEGER NOT NULL,
                caption_type TEXT NOT NULL,
                caption_text TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (deteccao_id) REFERENCES deteccoes (id)
            )
        ''')
        
        # Tabela para regiões densas do Florence-2
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS florence_dense_regions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                deteccao_id INTEGER NOT NULL,
                region_text TEXT,
                bbox_x1 REAL,
                bbox_y1 REAL,
                bbox_x2 REAL,
                bbox_y2 REAL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (deteccao_id) REFERENCES deteccoes (id)
            )
        ''')
        
        # Tabela para objetos detectados pelo Florence-2
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS florence_objects (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                deteccao_id INTEGER NOT NULL,
                object_label TEXT,
                bbox_x1 REAL,
                bbox_y1 REAL,
                bbox_x2 REAL,
                bbox_y2 REAL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (deteccao_id) REFERENCES deteccoes (id)
            )
        ''')
        
        # Tabela principal para resultados do GroundingDINO
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS grounding_dino_results (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                deteccao_id INTEGER NOT NULL,
                oil_spill_detected BOOLEAN NOT NULL,
                max_confidence REAL,
                total_detections INTEGER DEFAULT 0,
                image_width INTEGER,
                image_height INTEGER,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (deteccao_id) REFERENCES deteccoes (id)
            )
        ''')
        
        # Tabela para detecções individuais do GroundingDINO
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS grounding_detections (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                grounding_result_id INTEGER NOT NULL,
                phrase TEXT,
                confidence REAL,
                bbox_x1 REAL,
                bbox_y1 REAL,
                bbox_x2 REAL,
                bbox_y2 REAL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (grounding_result_id) REFERENCES grounding_dino_results (id)
            )
        ''')
        
        # Criar índices para melhor performance
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_deteccoes_coletor ON deteccoes(coletor_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_deteccoes_timestamp ON deteccoes(timestamp_coleta)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_deteccoes_modelo ON deteccoes(modelo_usado)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_florence_captions_deteccao ON florence_captions(deteccao_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_florence_captions_type ON florence_captions(caption_type)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_florence_regions_deteccao ON florence_dense_regions(deteccao_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_florence_objects_deteccao ON florence_objects(deteccao_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_grounding_results_deteccao ON grounding_dino_results(deteccao_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_grounding_detections_result ON grounding_detections(grounding_result_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_grounding_detections_confidence ON grounding_detections(confidence)')
        
        conn.commit()
        conn.close()
        logger.info(f"Normalized database initialized successfully for empresa: {empresa_id}")
    
    def insert_empresa(self, empresa_id: str, empresa_nome: str):
        """Insere ou atualiza dados da empresa em seu próprio banco"""
        self.init_database(empresa_id)  # Garante que o banco existe
        
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT OR REPLACE INTO empresa (empresa_id, empresa_nome, updated_at)
            VALUES (?, ?, CURRENT_TIMESTAMP)
        ''', (empresa_id, empresa_nome))
        
        conn.commit()
        conn.close()
    
    def insert_coletor(self, empresa_id: str, coletor_id: str, descricao: str, localizacao: str):
        """Insere ou atualiza coletor no banco da empresa"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT OR REPLACE INTO coletores 
            (coletor_id, coletor_descricao, coletor_localizacao, updated_at)
            VALUES (?, ?, ?, CURRENT_TIMESTAMP)
        ''', (coletor_id, descricao, localizacao))
        
        conn.commit()
        conn.close()
    
    def insert_deteccao(self, empresa_id: str, dados_deteccao: Dict) -> int:
        """Insere uma nova detecção no banco da empresa e retorna o ID"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO deteccoes 
            (coletor_id, timestamp_coleta, modelo_usado, 
             imagem_original_path, imagem_processada_path, confidence_yolo, objects_detected_yolo)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (
            dados_deteccao['coletor_id'],
            dados_deteccao['timestamp_coleta'],
            dados_deteccao['modelo_usado'],
            dados_deteccao['imagem_original_path'],
            dados_deteccao.get('imagem_processada_path'),
            dados_deteccao.get('confidence_yolo'),
            dados_deteccao.get('objects_detected_yolo', 0)
        ))
        
        deteccao_id = cursor.lastrowid
        conn.commit()
        conn.close()
        return deteccao_id
    
    def insert_florence_result(self, empresa_id: str, deteccao_id: int, florence_data: Dict):
        """Insere resultado do Florence-2 de forma normalizada no banco da empresa"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        # Inserir captions simples
        caption_types = {
            '<CAPTION>': 'caption',
            '<DETAILED_CAPTION>': 'detailed_caption',
            '<MORE_DETAILED_CAPTION>': 'more_detailed_caption'
        }
        
        for key, caption_type in caption_types.items():
            caption_text = florence_data.get(key, '')
            if caption_text:
                cursor.execute('''
                    INSERT INTO florence_captions (deteccao_id, caption_type, caption_text)
                    VALUES (?, ?, ?)
                ''', (deteccao_id, caption_type, caption_text))
        
        # Inserir regiões densas
        dense_regions = florence_data.get('<DENSE_REGION_CAPTION>', {})
        if isinstance(dense_regions, dict):
            labels = dense_regions.get('labels', [])
            bboxes = dense_regions.get('bboxes', [])
            
            for i, label in enumerate(labels):
                bbox = bboxes[i] if i < len(bboxes) else [0, 0, 0, 0]
                cursor.execute('''
                    INSERT INTO florence_dense_regions 
                    (deteccao_id, region_text, bbox_x1, bbox_y1, bbox_x2, bbox_y2)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (deteccao_id, label, bbox[0], bbox[1], bbox[2], bbox[3]))
        
        # Inserir objetos detectados
        objects_data = florence_data.get('<OD>', {})
        if isinstance(objects_data, dict):
            labels = objects_data.get('labels', [])
            bboxes = objects_data.get('bboxes', [])
            
            for i, label in enumerate(labels):
                bbox = bboxes[i] if i < len(bboxes) else [0, 0, 0, 0]
                cursor.execute('''
                    INSERT INTO florence_objects 
                    (deteccao_id, object_label, bbox_x1, bbox_y1, bbox_x2, bbox_y2)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (deteccao_id, label, bbox[0], bbox[1], bbox[2], bbox[3]))
        
        conn.commit()
        conn.close()
    
    def insert_grounding_result(self, empresa_id: str, deteccao_id: int, grounding_data: Dict):
        """Insere resultado do GroundingDINO de forma normalizada no banco da empresa"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        detection_data = grounding_data.get('detection_data', {})
        logits = detection_data.get('logits', [])
        boxes = detection_data.get('boxes', [])
        phrases = detection_data.get('phrases', [])
        image_dims = detection_data.get('image_dimensions', (0, 0))
        
        # Converter numpy arrays para listas Python se necessário
        if isinstance(logits, np.ndarray):
            logits_list = logits.tolist()
            has_detections = logits.size > 0
            max_confidence = float(np.max(logits)) if logits.size > 0 else 0.0
            total_detections = int(logits.size)
        else:
            logits_list = list(logits) if logits else []
            has_detections = len(logits_list) > 0
            max_confidence = float(max(logits_list)) if logits_list else 0.0
            total_detections = len(logits_list)
        
        if isinstance(boxes, np.ndarray):
            boxes_list = boxes.tolist()
        else:
            boxes_list = list(boxes) if boxes else []
        
        if isinstance(phrases, np.ndarray):
            phrases_list = phrases.tolist()
        else:
            phrases_list = list(phrases) if phrases else []
        
        # Inserir resultado principal do GroundingDINO
        cursor.execute('''
            INSERT INTO grounding_dino_results 
            (deteccao_id, oil_spill_detected, max_confidence, total_detections, image_width, image_height)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            deteccao_id,
            has_detections,
            max_confidence,
            total_detections,
            image_dims[1],  # width
            image_dims[0]   # height
        ))
        
        grounding_result_id = cursor.lastrowid
        
        # Inserir detecções individuais
        for i, confidence in enumerate(logits_list):
            phrase = phrases_list[i] if i < len(phrases_list) else ''
            bbox = boxes_list[i] if i < len(boxes_list) else [0, 0, 0, 0]
            
            cursor.execute('''
                INSERT INTO grounding_detections 
                (grounding_result_id, phrase, confidence, bbox_x1, bbox_y1, bbox_x2, bbox_y2)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (
                grounding_result_id,
                phrase,
                float(confidence),
                float(bbox[0]),
                float(bbox[1]),
                float(bbox[2]),
                float(bbox[3])
            ))
        
        conn.commit()
        conn.close()
    
    def get_detection_with_details(self, empresa_id: str, deteccao_id: int) -> Dict:
        """Recupera uma detecção completa com todos os detalhes normalizados do banco da empresa"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        # Buscar detecção principal com dados da empresa e coletor
        cursor.execute('''
            SELECT d.*, e.empresa_nome, c.coletor_descricao, c.coletor_localizacao
            FROM deteccoes d
            JOIN empresa e ON e.empresa_id = ?
            JOIN coletores c ON d.coletor_id = c.coletor_id
            WHERE d.id = ?
        ''', (empresa_id, deteccao_id))
        
        deteccao = cursor.fetchone()
        if not deteccao:
            conn.close()
            return {}
        
        # Buscar captions do Florence
        cursor.execute('''
            SELECT caption_type, caption_text
            FROM florence_captions
            WHERE deteccao_id = ?
        ''', (deteccao_id,))
        captions = {row[0]: row[1] for row in cursor.fetchall()}
        
        # Buscar regiões densas
        cursor.execute('''
            SELECT region_text, bbox_x1, bbox_y1, bbox_x2, bbox_y2
            FROM florence_dense_regions
            WHERE deteccao_id = ?
        ''', (deteccao_id,))
        dense_regions = [
            {
                'text': row[0],
                'bbox': [row[1], row[2], row[3], row[4]]
            }
            for row in cursor.fetchall()
        ]
        
        # Buscar objetos do Florence
        cursor.execute('''
            SELECT object_label, bbox_x1, bbox_y1, bbox_x2, bbox_y2
            FROM florence_objects
            WHERE deteccao_id = ?
        ''', (deteccao_id,))
        florence_objects = [
            {
                'label': row[0],
                'bbox': [row[1], row[2], row[3], row[4]]
            }
            for row in cursor.fetchall()
        ]
        
        # Buscar resultados do GroundingDINO
        cursor.execute('''
            SELECT oil_spill_detected, max_confidence, total_detections, image_width, image_height
            FROM grounding_dino_results
            WHERE deteccao_id = ?
        ''', (deteccao_id,))
        grounding_main = cursor.fetchone()
        
        # Buscar detecções individuais do GroundingDINO
        cursor.execute('''
            SELECT gd.phrase, gd.confidence, gd.bbox_x1, gd.bbox_y1, gd.bbox_x2, gd.bbox_y2
            FROM grounding_detections gd
            JOIN grounding_dino_results gdr ON gd.grounding_result_id = gdr.id
            WHERE gdr.deteccao_id = ?
            ORDER BY gd.confidence DESC
        ''', (deteccao_id,))
        grounding_detections = [
            {
                'phrase': row[0],
                'confidence': row[1],
                'bbox': [row[2], row[3], row[4], row[5]]
            }
            for row in cursor.fetchall()
        ]
        
        conn.close()
        
        # Montar resultado completo
        return {
            'deteccao': {
                'id': deteccao[0],
                'empresa_id': empresa_id,
                'empresa_nome': deteccao[9],
                'coletor_id': deteccao[1],
                'coletor_descricao': deteccao[10],
                'coletor_localizacao': deteccao[11],
                'timestamp_coleta': deteccao[2],
                'timestamp_processamento': deteccao[3],
                'modelo_usado': deteccao[4],
                'imagem_original_path': deteccao[5],
                'imagem_processada_path': deteccao[6],
                'confidence_yolo': deteccao[7],
                'objects_detected_yolo': deteccao[8]
            },
            'florence': {
                'captions': captions,
                'dense_regions': dense_regions,
                'objects': florence_objects
            },
            'grounding_dino': {
                'oil_spill_detected': grounding_main[0] if grounding_main else False,
                'max_confidence': grounding_main[1] if grounding_main else 0.0,
                'total_detections': grounding_main[2] if grounding_main else 0,
                'image_dimensions': [grounding_main[4], grounding_main[3]] if grounding_main else [0, 0],
                'detections': grounding_detections
            }
        }
    
    def get_empresa_stats(self, empresa_id: str) -> Dict:
        """Retorna estatísticas específicas da empresa"""
        try:
            db_path = self.get_db_path(empresa_id)
            if not db_path.exists():
                return {"error": "Database not found for empresa"}
                
            conn = sqlite3.connect(str(db_path))
            cursor = conn.cursor()
            
            # Estatísticas gerais
            cursor.execute("SELECT COUNT(*) FROM deteccoes")
            total_deteccoes = cursor.fetchone()[0]
            
            cursor.execute("SELECT COUNT(*) FROM grounding_dino_results WHERE oil_spill_detected = 1")
            deteccoes_positivas = cursor.fetchone()[0]
            
            cursor.execute("SELECT COUNT(DISTINCT coletor_id) FROM deteccoes")
            total_coletores = cursor.fetchone()[0]
            
            # Top coletores com mais detecções
            cursor.execute('''
                SELECT c.coletor_id, c.coletor_descricao, COUNT(d.id) as total_deteccoes
                FROM coletores c
                LEFT JOIN deteccoes d ON c.coletor_id = d.coletor_id
                GROUP BY c.coletor_id
                ORDER BY total_deteccoes DESC
                LIMIT 5
            ''')
            top_coletores = cursor.fetchall()
            
            conn.close()
            
            return {
                "empresa_id": empresa_id,
                "total_deteccoes": total_deteccoes,
                "deteccoes_positivas": deteccoes_positivas,
                "taxa_deteccao": (deteccoes_positivas / total_deteccoes * 100) if total_deteccoes > 0 else 0,
                "total_coletores": total_coletores,
                "top_coletores": [
                    {
                        "coletor_id": row[0],
                        "descricao": row[1],
                        "total_deteccoes": row[2]
                    }
                    for row in top_coletores
                ]
            }
            
        except Exception as e:
            logger.error(f"Erro ao obter estatísticas da empresa {empresa_id}: {e}")
            return {"error": str(e)}


class FileManager:
    """Gerenciador de arquivos e diretórios por empresa"""
    
    def __init__(self, base_path: str = "Overseas"):
        self.base_path = Path(base_path)
        self.base_path.mkdir(exist_ok=True)
    
    def create_directory_structure(self, empresa_id: str, coletor_id: str, modelo: str) -> Tuple[Path, Path]:
        """Cria estrutura de diretórios específica da empresa e retorna paths para raw e processed"""
        base_dir = self.base_path / empresa_id / coletor_id / modelo
        raw_dir = base_dir / "raw_image"
        processed_dir = base_dir / "processed"
        
        raw_dir.mkdir(parents=True, exist_ok=True)
        processed_dir.mkdir(parents=True, exist_ok=True)
        
        return raw_dir, processed_dir
    
    def generate_filename(self, empresa_id: str, coletor_id: str, timestamp: str, 
                         modelo: str, objects_count: int, prefix: str = "RAW") -> str:
        """Gera nome do arquivo seguindo o padrão especificado"""
        return f"{prefix}-{empresa_id}-{coletor_id}-{timestamp}-{modelo}-{objects_count}.jpg"
    
    def save_image(self, image: Image.Image, filepath: Path) -> bool:
        """Salva imagem no caminho especificado"""
        try:
            image.save(filepath, "JPEG", quality=95)
            return True
        except Exception as e:
            logger.error(f"Erro ao salvar imagem {filepath}: {e}")
            return False
    
    def get_empresa_directory_info(self, empresa_id: str) -> Dict:
        """Retorna informações sobre os diretórios da empresa"""
        empresa_dir = self.base_path / empresa_id
        if not empresa_dir.exists():
            return {"error": "Empresa directory not found"}
        
        info = {
            "empresa_id": empresa_id,
            "base_path": str(empresa_dir),
            "coletores": []
        }
        
        for coletor_dir in empresa_dir.iterdir():
            if coletor_dir.is_dir():
                coletor_info = {
                    "coletor_id": coletor_dir.name,
                    "path": str(coletor_dir),
                    "modelos": []
                }
                
                for modelo_dir in coletor_dir.iterdir():
                    if modelo_dir.is_dir():
                        raw_count = len(list((modelo_dir / "raw_image").glob("*.jpg"))) if (modelo_dir / "raw_image").exists() else 0
                        processed_count = len(list((modelo_dir / "processed").glob("*.jpg"))) if (modelo_dir / "processed").exists() else 0
                        
                        coletor_info["modelos"].append({
                            "modelo": modelo_dir.name,
                            "raw_images": raw_count,
                            "processed_images": processed_count
                        })
                
                info["coletores"].append(coletor_info)
        
        return info
    
class ModelManager:
    """Gerenciador dos modelos Florence-2 e GroundingDINO"""
    
    def __init__(self):
        self.florence_processor = None
        self.florence_model = None
        self.grounding_model = None
        self._load_models()
    
    def _load_models(self):
        """Carrega os modelos na inicialização"""
        try:
            # Carregar Florence-2
            logger.info("Loading Florence-2 model...")
            from transformers import AutoProcessor, AutoModelForCausalLM
            
            self.florence_processor = AutoProcessor.from_pretrained(
                "microsoft/florence-2-base-ft", trust_remote_code=True
            )
            self.florence_model = AutoModelForCausalLM.from_pretrained(
                "microsoft/florence-2-base-ft", trust_remote_code=True
            )
            logger.info("Florence-2 model loaded successfully")
            
            # Carregar GroundingDINO (comentado por enquanto - você precisa ajustar os imports)
            logger.info("Loading GroundingDINO model...")
            self.grounding_model = load_model(CONFIG_PATH, WEIGHTS_PATH, device='cpu')
            logger.info("GroundingDINO model loaded successfully")
            
        except Exception as e:
            logger.error(f"Erro ao carregar modelos: {e}")
    
    def generate_florence_captions(self, image: Image.Image, max_new_tokens: int = 1024) -> Dict:
        """Gera captions usando Florence-2"""
        if not self.florence_model or not self.florence_processor:
            logger.warning("Florence-2 model not loaded, returning empty results")
            return {}
        
        prompts = [
            "<CAPTION>",
            "<DETAILED_CAPTION>", 
            "<MORE_DETAILED_CAPTION>",
            "<DENSE_REGION_CAPTION>",
            "<OD>",
        ]
        
        results = {}
        for prompt in prompts:
            try:
                inputs = self.florence_processor(text=prompt, images=image, return_tensors="pt")
                generated_ids = self.florence_model.generate(
                    input_ids=inputs["input_ids"],
                    pixel_values=inputs["pixel_values"],
                    max_new_tokens=max_new_tokens
                )
                generated_text = self.florence_processor.batch_decode(
                    generated_ids, skip_special_tokens=False
                )[0]
                parsed_answer = self.florence_processor.post_process_generation(
                    generated_text,
                    task=prompt,
                    image_size=(image.width, image.height)
                )
                results[prompt] = parsed_answer.get(prompt, "")
            except Exception as e:
                logger.error(f"Erro no Florence-2 com prompt {prompt}: {e}")
                results[prompt] = ""
        
        return results
    
    def process_grounding_detection(self, image: Image.Image, 
                              text_prompt: str = "oil spill",
                              box_threshold: float = 0.75,
                              text_threshold: float = 0.5) -> Dict:
        """Processa detecção com GroundingDINO"""
        try:
            if not self.grounding_model:
                logger.warning("GroundingDINO model not loaded, returning empty results")
                return {
                    'annotated_image': np.array(image),
                    'detection_data': {
                        'boxes': np.array([]),
                        'logits': np.array([]),
                        'phrases': [],
                        'image_dimensions': (image.height, image.width)
                    }
                }
            
            # Converter PIL Image para numpy array
            image_array = np.array(image)
            
            # Converter imagem para tensor
            image_tensor = torch.from_numpy(image_array).permute(2, 0, 1).float()
            
            # Processar detecção
            self.grounding_model.eval()
            with torch.no_grad():
                boxes, logits, phrases = predict(
                    model=self.grounding_model,
                    image=image_tensor,
                    caption=text_prompt,
                    box_threshold=box_threshold,
                    text_threshold=text_threshold,
                    device='cpu'
                )
            
            # Converter tensores para numpy
            boxes_np = boxes.cpu().numpy()
            logits_np = logits.cpu().numpy()
            
            # Anotar a imagem
            annotated_frame = annotate(image_array, boxes, logits, phrases)
            
            return {
                'annotated_image': annotated_frame,
                'detection_data': {
                    'boxes': boxes_np,
                    'logits': logits_np,
                    'phrases': phrases,
                    'image_dimensions': (image.height, image.width)
                }
            }
            
        except Exception as e:
            logger.error(f"Erro no GroundingDINO: {e}")
            # Retornar estrutura vazia em caso de erro
            return {
                'annotated_image': np.array(image),
                'detection_data': {
                    'boxes': np.array([]),
                    'logits': np.array([]),
                    'phrases': [],
                    'image_dimensions': (image.height, image.width)
                }
            }


In [ ]:
## 🎯 OBJETIVO PRINCIPAL

Desenvolver uma classe WebDashboard completa e profissional para visualização e análise de dados de detecção de vazamento de óleo em tempo real, integrada a um sistema Flask existente.

## 📋 ESPECIFICAÇÕES TÉCNICAS OBRIGATÓRIAS

### Estrutura Base da Classe

```python

class WebDashboard:

    """Classe para interface web do sistema de detecção de óleo"""

    def init(self, db_manager, file_manager):

        self.db_manager = db_manager

        self.file_manager = file_manager

```

### Integração com Sistema Existente

- A classe será instanciada dentro de OilDetectionServer

- Deve implementar método setup_routes(app) para configurar rotas Flask

- Deve implementar método get_dashboard_template() para página principal

- Execução em localhost via Flask

## 🗄️ ESTRUTURA DE DADOS

### Localização dos Bancos

```

Overseas/ID_EMPRESA/ID_EMPRESA_detection.db

```

- Múltiplas empresas no diretório Overseas

- Cada empresa possui seu próprio banco SQLite

### Esquema Completo das Tabelas (Dados Reais do Sistema):

```python

# DADOS REAIS DO BANCO - USE ESTES COMO REFERÊNCIA:

empresa: {

    'empresa_id': {0: 'DOS0724'}, 

    'empresa_nome': {0: 'Data Overseas'}, 

    'created_at': {0: '2025-06-14 22:35:48'}, 

    'updated_at': {0: '2025-06-14 22:35:48'}

}

coletores: {

    'coletor_id': {0: 'APS005'}, 

    'coletor_descricao': {0: 'Camera Costeiro - Santos'}, 

    'coletor_localizacao': {0: '-23.9608, -46.3336'}, 

    'status': {0: 'ativo'}, 

    'created_at': {0: '2025-06-14 22:35:48'}, 

    'updated_at': {0: '2025-06-14 22:35:48'}

}

deteccoes: {

    'id': {0: 1, 1: 2}, 

    'coletor_id': {0: 'APS005', 1: 'APS005'}, 

    'timestamp_coleta': {0: '2025-06-14T18:51:15.315952', 1: '2025-06-14T19:35:39.833573'}, 

    'timestamp_processamento': {0: '2025-06-14 21:51:36', 1: '2025-06-14 22:36:00'}, 

    'modelo_usado': {0: 'oil_spill', 1: 'oil_spill'}, 

    'imagem_original_path': {0: 'Overseas\\DOS0724\\APS005\\oil_spill\\raw_image\\RAW-DOS0724-APS005-oil_spill-20250614_185115_315-1.png', 1: 'Overseas\\DOS0724\\APS005\\oil_spill\\raw_image\\RAW-DOS0724-APS005-oil_spill-20250614_193539_833-1.png'}, 

    'imagem_processada_path': {0: 'Overseas\\DOS0724\\APS005\\oil_spill\\processed\\PRC-DOS0724-APS005-oil_spill-20250614_185115_315-1.png', 1: 'Overseas\\DOS0724\\APS005\\oil_spill\\processed\\PRC-DOS0724-APS005-oil_spill-20250614_193539_833-1.png'}, 

    'confidence_yolo': {0: 0.724217, 1: 0.856465}, 

    'objects_detected_yolo': {0: 1, 1: 1}

}

sqlite_sequence: {

    'name': {0: 'deteccoes', 1: 'florence_captions'}, 

    'seq': {0: 2, 1: 6}

}

florence_captions: {

    'id': {0: 1, 1: 2}, 

    'deteccao_id': {0: 1, 1: 1}, 

    'caption_type': {0: 'caption', 1: 'detailed_caption'}, 

    'caption_text': {0: 'A person sitting in front of a white wall.', 1: "In this image we can see a person's leg."}, 

    'created_at': {0: '2025-06-14 21:51:36', 1: '2025-06-14 21:51:36'}

}

florence_dense_regions: {

    'id': {0: 1}, 

    'deteccao_id': {0: 1}, 

    'region_text': {0: 'person'}, 

    'bbox_x1': {0: 0.3199999928474426}, 

    'bbox_y1': {0: 26.15999984741211}, 

    'bbox_x2': {0: 612.7999877929688}, 

    'bbox_y2': {0: 479.2799987792969}, 

    'created_at': {0: '2025-06-14 21:51:36'}

}

florence_objects: {

    'id': {0: 1, 1: 2}, 

    'deteccao_id': {0: 1, 1: 1}, 

    'object_label': {0: 'footwear', 1: 'person'}, 

    'bbox_x1': {0: 0.3199999928474426, 1: 0.3199999928474426}, 

    'bbox_y1': {0: 26.15999984741211, 1: 26.15999984741211}, 

    'bbox_x2': {0: 31.03999900817871, 1: 639.0399780273438}, 

    'bbox_y2': {0: 97.68000030517578, 1: 479.2799987792969}, 

    'created_at': {0: '2025-06-14 21:51:36', 1: '2025-06-14 21:51:36'}

}

grounding_dino_results: {

    'id': {0: 1, 1: 2}, 

    'deteccao_id': {0: 1, 1: 2}, 

    'oil_spill_detected': {0: 1, 1: 1}, 

    'max_confidence': {0: 0.43636131286621094, 1: 0.7241383790969849}, 

    'total_detections': {0: 2, 1: 1}, 

    'image_width': {0: 640, 1: 640}, 

    'image_height': {0: 480, 1: 480}, 

    'created_at': {0: '2025-06-14 21:51:36', 1: '2025-06-14 22:36:00'}

}

grounding_detections: {

    'id': {0: 1, 1: 2}, 

    'grounding_result_id': {0: 1, 1: 1}, 

    'phrase': {0: 'oil spill [SEP]', 1: 'oil spill [SEP]'}, 

    'confidence': {0: 0.43636131286621094, 1: 0.35062718391418457}, 

    'bbox_x1': {0: 0.308290034532547, 1: 0.49945348501205444}, 

    'bbox_y1': {0: 0.5970625281333923, 1: 0.49977728724479675}, 

    'bbox_x2': {0: 0.6144207119941711, 1: 0.9938387274742126}, 

    'bbox_y2': {0: 0.7960148453712463, 1: 0.9921114444732666}, 

    'created_at': {0: '2025-06-14 21:51:36', 1: '2025-06-14 21:51:36'}

}

```

### Análise Crítica dos Dados:

- ATENÇÃO: No exemplo, grounding_dino_results mostra max_confidence de 0.436 (43.6%) e 0.724 (72.4%)

- REGRA: Apenas a segunda detecção (72.4%) deve ser considerada válida (>70%)

- CORRELAÇÃO: deteccao_id conecta todas as tabelas de análise

- TEMPORAL: Timestamps mostram coleta às 18:51 e 19:35, processamento às 21:51 e 22:36

## 🧠 REGRAS DE NEGÓCIO CRÍTICAS

### 1. Análise Temporal Inteligente

- NUNCA contar detecções como eventos únicos

- Analisar períodos de presença de vazamento (de tempo X até tempo Y)

- Agrupar detecções consecutivas como um único evento temporal

### 2. Filtro de Confiabilidade

- Threshold mínimo: 70% de confiança para considerar detecção válida

- Destacar visualmente detecções com baixa confiança (< 70%)

- Permitir ajuste dinâmico do threshold

### 3. Análise Multi-Modelo

- Combinar resultados de YOLO, Florence e Grounding DINO

- Correlacionar dados entre modelos para maior precisão

- Mostrar consenso entre modelos

## 🎨 REQUISITOS DE INTERFACE

### Design Moderno e Responsivo

- Tema: Escuro/industrial adequado para monitoramento

- Cores: Azul/cinza para normal, vermelho para alertas

- Layout: Dashboard estilo NOC (Network Operations Center)

### Componentes Obrigatórios

#### 1. Seletor de Empresa Dinâmico

- Dropdown para alternar entre empresas

- Carregamento automático dos bancos disponíveis

#### 2. Painel de Status em Tempo Real

- Status de cada coletor (ativo/inativo)

- Última detecção por coletor

- Indicadores visuais de saúde do sistema

#### 3. Mapa Interativo

- Plotar coletores por localização GPS

- Marcadores coloridos por status

- Popup com informações detalhadas

#### 4. Timeline de Eventos

- Linha temporal de vazamentos detectados

- Duração estimada de cada evento

- Filtros por período e coletor

#### 5. Análise Estatística Avançada

- Gráficos de tendência de confiança

- Comparação entre modelos de IA

- Métricas de performance por coletor

#### 6. Galeria de Imagens

- Visualização de imagens originais vs processadas

- Zoom e anotações das detecções

- Navegação temporal das capturas

#### 7. Sistema de Alertas

- Alertas para detecções de alta confiança

- Notificações de coletores offline

- Log de eventos críticos

## 💻 IMPLEMENTAÇÃO REQUERIDA

### Métodos Essenciais

```python

def setup_routes(self, app):

    """Configurar todas as rotas Flask necessárias"""

    

def get_dashboard_template(self):

    """Retornar template HTML principal do dashboard"""

    

def get_companies_list(self):

    """Listar empresas disponíveis dinamicamente"""

    

def get_company_data(self, company_id):

    """Obter dados completos de uma empresa"""

    

def analyze_temporal_events(self, detections):

    """Analisar eventos temporais de vazamento"""

    

def get_filtered_detections(self, confidence_threshold=0.7):

    """Filtrar detecções por confiabilidade"""

```

### Tecnologias a Utilizar

- Backend: Flask, SQLite3

- Frontend: HTML5, CSS3, JavaScript (vanilla ou framework leve)

- Visualização: Chart.js, Leaflet (mapas)

- UI: Bootstrap ou framework similar

## 🚀 CRITÉRIOS DE EXCELÊNCIA

### Performance

- Consultas SQL otimizadas

- Cache inteligente de dados

- Carregamento assíncrono

### Usabilidade

- Interface intuitiva para operadores

- Responsividade mobile

- Acessibilidade (WCAG)

### Robustez

- Tratamento de erros gracioso

- Validação de dados

- Fallbacks para dados ausentes

## ⚠️ RESTRIÇÕES E CONSIDERAÇÕES

1. NÃO usar dados estáticos/hardcoded

2. SEMPRE consultar banco de dados dinamicamente  

3. CONSIDERAR cenários de múltiplas empresas

4. IMPLEMENTAR análise temporal inteligente

5. PRIORIZAR usabilidade em ambiente operacional

## 📊 ENTREGÁVEL ESPERADO

Uma classe WebDashboard completa, profissional e production-ready que:

- Integre seamlessly com o sistema Flask existente

- Forneça visualização rica e interativa dos dados

- Implemente todas as regras de negócio especificadas

- Seja extensível para futuras funcionalidades

- Demonstre expertise em data science e desenvolvimento web

IMPORTANTE: Esta é uma aplicação crítica para monitoramento ambiental. A qualidade, precisão e confiabilidade são fundamentais.

---

## 🔥 INSTRUÇÕES PARA MÁXIMA PERFORMANCE DA IA

### Para a IA que receberá este prompt:

CONTEXTO: Você é um desenvolvedor sênior especializado em dashboards industriais e análise de dados ambientais. Este não é um projeto acadêmico - é um sistema real de monitoramento de vazamento de óleo que será usado em operações críticas.

EXPECTATIVAS:

1. SER PROATIVO: Não se limite ao mínimo. Implemente funcionalidades que um usuário real precisaria

2. PENSAR COMO DATA SCIENTIST: Considere patterns, outliers, correlações nos dados

3. QUALIDADE INDUSTRIAL: Código limpo, comentado, com tratamento de erro

4. UX/UI PROFISSIONAL: Interface que operators reais usariam 24/7

NÃO FAÇA:

- Templates básicos com dados fake

- Código incompleto ou "placeholder"

- Interface genérica sem contexto do domínio

- Análises superficiais dos dados

FAÇA:

- Dashboard completo e funcional

- Análise temporal inteligente real

- Interface específica para monitoramento de óleo

- Integração robusta com Flask existente

- Consultas SQL otimizadas e dinâmicas

RESULTADO ESPERADO: Código production-ready que impressione pela completude e atenção aos detalhes específicos do domínio de monitoramento ambiental.

---

## 📝 FORMATO DA RESPOSTA ESPERADA

ENTREGUE APENAS:

```python

class WebDashboard:

    """Classe para interface web do sistema de detecção de óleo"""

    def init(self, db_manager, file_manager):

        self.db_manager = db_manager

        self.file_manager = file_manager

        # ... implementação completa

    

    def setup_routes(self, app):

        """Configurar todas as rotas Flask necessárias"""

        # ... implementação completa

    

    def get_dashboard_template(self):

        """Retornar template HTML principal do dashboard"""

        # ... implementação completa

    

    # ... todos os outros métodos necessários

```

NÃO INCLUA:

- Explicações longas antes do código

- Código de exemplo de como usar

- Imports separados (inclua dentro da classe se necessário)

- Documentação adicional

FOQUE EM:

- Código limpo e comentado

- Implementação completa de todos os métodos

- HTML/CSS/JS embutido quando necessário

- Consultas SQL dinâmicas para os dados reais

- Interface visual profissional e funcional

LEMBRE-SE: O usuário quer apenas a classe WebDashboard pronta para integrar no sistema existente. Seja direto e completo.

EStrutura para você continuar a partir disso:

class WebDashboard:

    """Classe para interface web do sistema de detecção de óleo"""

    

    def init(self, db_manager, file_manager):

        self.db_manager = db_manager

        self.file_manager = file_manager

        self.base_path = Path("Overseas")

    

    def setup_routes(self, app):

        """Configura as rotas do dashboard"""

        

        @app.route('/web-dashboard')

        def web_dashboard():

            return self.get_dashboard_template()

        

        @app.route('/api/empresas')

        def api_get_empresas():

            return jsonify(self.get_empresas_list())

        

        @app.route('/api/empresa/<empresa_id>/dados')

        def api_get_empresa_dados(empresa_id):

            return jsonify(self.get_empresa_dados(empresa_id))

        

        @app.route('/api/empresa/<empresa_id>/deteccao/<int:deteccao_id>')

        def api_get_deteccao_detalhes(empresa_id, deteccao_id):

            return jsonify(self.get_deteccao_detalhes(empresa_id, deteccao_id))

    

    def get_empresas_list(self):

        """Lista todas as empresas disponíveis"""

        empresas = []

        if self.base_path.exists():

            for empresa_dir in self.base_path.iterdir():

                if empresa_dir.is_dir():

                    db_path = empresa_dir / f"{empresa_dir.name}_detection.db"

                    if db_path.exists():

                        empresas.append({

                            'id': empresa_dir.name,

                            'path': str(db_path)

                        })

        return empresas

    

    def get_deteccao_detalhes(self, empresa_id, deteccao_id):

        """Obtém detalhes completos de uma detecção específica"""

        db_path = self.base_path / empresa_id / f"{empresa_id}_detection.db"

        

        if not db_path.exists():

            return {"error": "Banco de dados não encontrado"}

        

        try:

            conn = sqlite3.connect(str(db_path))

            conn.row_factory = sqlite3.Row

            cursor = conn.cursor()

            

            # Detecção principal

            cursor.execute("SELECT * FROM deteccoes WHERE id = ?", (deteccao_id,))

            deteccao = dict(cursor.fetchone())

            

            # Florence captions

            cursor.execute("SELECT * FROM florence_captions WHERE deteccao_id = ?", (deteccao_id,))

            captions = [dict(row) for row in cursor.fetchall()]

            

            # Florence objects

            cursor.execute("SELECT * FROM florence_objects WHERE deteccao_id = ?", (deteccao_id,))

            objects = [dict(row) for row in cursor.fetchall()]

            

            # Grounding DINO results

            cursor.execute("SELECT * FROM grounding_dino_results WHERE deteccao_id = ?", (deteccao_id,))

            grounding_result = cursor.fetchone()

            grounding_result = dict(grounding_result) if grounding_result else {}

            

            # Grounding detections

            if grounding_result:

                cursor.execute("SELECT * FROM grounding_detections WHERE grounding_result_id = ?", 

                             (grounding_result.get('id'),))

                grounding_detections = [dict(row) for row in cursor.fetchall()]

            else:

                grounding_detections = []

            

            conn.close()

            

            return {

                "deteccao": deteccao,

                "florence_captions": captions,

                "florence_objects": objects,

                "grounding_result": grounding_result,

                "grounding_detections": grounding_detections

            }

            

        except Exception as e:

            return {"error": str(e)}

    

    def get_empresa_dados(self, empresa_id):

        """Obtém todos os dados de uma empresa com análises avançadas"""

        db_path = self.base_path / empresa_id / f"{empresa_id}_detection.db"

        

        if not db_path.exists():

            return {"error": "Banco de dados não encontrado"}

        

        try:

            conn = sqlite3.connect(str(db_path))

            conn.row_factory = sqlite3.Row

            cursor = conn.cursor()

            

            # Buscar detecções com joins completos

            deteccoes_query = """

            SELECT d.*, g.max_confidence, g.oil_spill_detected, g.total_detections,

                   g.image_width, g.image_height

            FROM deteccoes d

            LEFT JOIN grounding_dino_results g ON d.id = g.deteccao_id

            ORDER BY d.timestamp_coleta DESC

            """

            

            cursor.execute(deteccoes_query)

            deteccoes = [dict(row) for row in cursor.fetchall()]

            

            # Buscar dados da empresa

            cursor.execute("SELECT * FROM empresa LIMIT 1")

            empresa_row = cursor.fetchone()

            empresa = dict(empresa_row) if empresa_row else {}

            

            # Buscar coletores

            cursor.execute("SELECT * FROM coletores")

            coletores = [dict(row) for row in cursor.fetchall()]

            

            # Análises avançadas

            stats = self.calculate_advanced_stats(deteccoes, cursor)

            

            # Dados para gráficos temporais

            timeline_data = self.get_timeline_data(deteccoes)

            

            # Análise de confiança

            confidence_analysis = self.analyze_confidence_distribution(deteccoes)

            

            # Análise por coletor

            collector_analysis = self.analyze_by_collector(deteccoes)

            

            conn.close()

            

            return {

                "empresa": empresa,

                "coletores": coletores,

                "deteccoes": deteccoes,

                "stats": stats,

                "timeline_data": timeline_data,

                "confidence_analysis": confidence_analysis,

                "collector_analysis": collector_analysis

            }

            

        except Exception as e:

            return {"error": str(e)}

    

    def calculate_advanced_stats(self, deteccoes, cursor):

        """Calcula estatísticas avançadas"""

        total_deteccoes = len(deteccoes)

        deteccoes_positivas = sum(1 for d in deteccoes if d.get('oil_spill_detected', 0) == 1)

        

        # Análise de confiança

        confidences = [d.get('max_confidence', 0) for d in deteccoes if d.get('max_confidence')]

        avg_confidence = sum(confidences) / len(confidences) if confidences else 0

        

        # Análise YOLO vs Grounding DINO

        yolo_detections = sum(d.get('objects_detected_yolo', 0) for d in deteccoes)

        grounding_detections = sum(d.get('total_detections', 0) for d in deteccoes)

        

        # Análise temporal (últimas 24h, 7 dias, 30 dias)

        from datetime import datetime, timedelta

        now = datetime.now()

        

        recent_24h = sum(1 for d in deteccoes 

                        if self.parse_timestamp(d.get('timestamp_coleta', '')) > now - timedelta(hours=24))

        recent_7d = sum(1 for d in deteccoes 

                       if self.parse_timestamp(d.get('timestamp_coleta', '')) > now - timedelta(days=7))

        

        # Contagem de captions Florence

        cursor.execute("SELECT COUNT(*) as total FROM florence_captions")

        total_captions = cursor.fetchone()[0]

        

        # Contagem de objetos Florence

        cursor.execute("SELECT COUNT(*) as total FROM florence_objects")

        total_objects = cursor.fetchone()[0]

        

        return {

            "total_deteccoes": total_deteccoes,

            "deteccoes_positivas": deteccoes_positivas,

            "taxa_deteccao": round((deteccoes_positivas / total_deteccoes * 100) if total_deteccoes > 0 else 0, 2),

            "confianca_media": round(avg_confidence * 100, 2),

            "yolo_detections": yolo_detections,

            "grounding_detections": grounding_detections,

            "recent_24h": recent_24h,

            "recent_7d": recent_7d,

            "total_captions": total_captions,

            "total_objects": total_objects,

            "confidence_ranges": {

                "high": sum(1 for c in confidences if c >= 0.7),

                "medium": sum(1 for c in confidences if 0.5 <= c < 0.7),

                "low": sum(1 for c in confidences if c < 0.5)

            }

        }

    

    def parse_timestamp(self, timestamp_str):

        """Parse timestamp string to datetime"""

        try:

            return datetime.fromisoformat(timestamp_str.replace('T', ' ').split('.')[0])

        except:

            return datetime.min

    

    def get_timeline_data(self, deteccoes):

        """Prepara dados para gráfico temporal"""

        from collections import defaultdict

        

        daily_counts = defaultdict(lambda: {"total": 0, "positivas": 0})

        

        for deteccao in deteccoes:

            try:

                dt = self.parse_timestamp(deteccao.get('timestamp_coleta', ''))

                date_key = dt.strftime('%Y-%m-%d')

                daily_counts[date_key]["total"] += 1

                if deteccao.get('oil_spill_detected', 0) == 1:

                    daily_counts[date_key]["positivas"] += 1

            except:

                continue

        

        return dict(daily_counts)

    

    def analyze_confidence_distribution(self, deteccoes):

        """Analisa distribuição de confiança"""

        confidences = [d.get('max_confidence', 0) for d in deteccoes if d.get('max_confidence')]

        

        if not confidences:

            return {"ranges": {}, "average": 0, "max": 0, "min": 0}

        

        ranges = {

            "0-20%": sum(1 for c in confidences if 0 <= c < 0.2),

            "20-40%": sum(1 for c in confidences if 0.2 <= c < 0.4),

            "40-60%": sum(1 for c in confidences if 0.4 <= c < 0.6),

            "60-80%": sum(1 for c in confidences if 0.6 <= c < 0.8),

            "80-100%": sum(1 for c in confidences if 0.8 <= c <= 1.0)

        }

        

        return {

            "ranges": ranges,

            "average": round(sum(confidences) / len(confidences) * 100, 2),

            "max": round(max(confidences) * 100, 2),

            "min": round(min(confidences) * 100, 2)

        }

    

    def analyze_by_collector(self, deteccoes):

        """Análise por coletor"""

        from collections import defaultdict

        

        collector_stats = defaultdict(lambda: {"total": 0, "positivas": 0, "avg_confidence": 0, "confidences": []})

        

        for deteccao in deteccoes:

            coletor = deteccao.get('coletor_id', 'Unknown')

            collector_stats[coletor]["total"] += 1

            

            if deteccao.get('oil_spill_detected', 0) == 1:

                collector_stats[coletor]["positivas"] += 1

            

            if deteccao.get('max_confidence'):

                collector_stats[coletor]["confidences"].append(deteccao['max_confidence'])

        

        # Calcular médias

        for coletor, stats in collector_stats.items():

            if stats["confidences"]:

                stats["avg_confidence"] = round(sum(stats["confidences"]) / len(stats["confidences"]) * 100, 2)

            stats["taxa_deteccao"] = round((stats["positivas"] / stats["total"] * 100) if stats["total"] > 0 else 0, 2)

            del stats["confidences"]  # Remove lista para JSON

        

        return dict(collector_stats)

    def get_dashboard_template(self):

        """Retorna o template HTML melhorado da dashboard"""

        html_template = """

In [37]:
class WebDashboard:
    """Classe para interface web do sistema de detecção de óleo"""
    
    def __init__(self, db_manager, file_manager):
        self.db_manager = db_manager
        self.file_manager = file_manager
        self.base_path = Path("Overseas")
    
    def setup_routes(self, app):
        """Configura as rotas do dashboard"""
        
        @app.route('/web-dashboard')
        def web_dashboard():
            return self.get_dashboard_template()
        
        @app.route('/api/empresas')
        def api_get_empresas():
            return jsonify(self.get_empresas_list())
        
        @app.route('/api/empresa/<empresa_id>/dados')
        def api_get_empresa_dados(empresa_id):
            return jsonify(self.get_empresa_dados(empresa_id))
        
        @app.route('/api/empresa/<empresa_id>/deteccao/<int:deteccao_id>')
        def api_get_deteccao_detalhes(empresa_id, deteccao_id):
            return jsonify(self.get_deteccao_detalhes(empresa_id, deteccao_id))
    
    def get_empresas_list(self):
        """Lista todas as empresas disponíveis"""
        empresas = []
        if self.base_path.exists():
            for empresa_dir in self.base_path.iterdir():
                if empresa_dir.is_dir():
                    db_path = empresa_dir / f"{empresa_dir.name}_detection.db"
                    if db_path.exists():
                        empresas.append({
                            'id': empresa_dir.name,
                            'path': str(db_path)
                        })
        return empresas
    
    def get_deteccao_detalhes(self, empresa_id, deteccao_id):
        """Obtém detalhes completos de uma detecção específica"""
        db_path = self.base_path / empresa_id / f"{empresa_id}_detection.db"
        
        if not db_path.exists():
            return {"error": "Banco de dados não encontrado"}
        
        try:
            conn = sqlite3.connect(str(db_path))
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            # Detecção principal
            cursor.execute("SELECT * FROM deteccoes WHERE id = ?", (deteccao_id,))
            deteccao = dict(cursor.fetchone())
            
            # Florence captions
            cursor.execute("SELECT * FROM florence_captions WHERE deteccao_id = ?", (deteccao_id,))
            captions = [dict(row) for row in cursor.fetchall()]
            
            # Florence objects
            cursor.execute("SELECT * FROM florence_objects WHERE deteccao_id = ?", (deteccao_id,))
            objects = [dict(row) for row in cursor.fetchall()]
            
            # Grounding DINO results
            cursor.execute("SELECT * FROM grounding_dino_results WHERE deteccao_id = ?", (deteccao_id,))
            grounding_result = cursor.fetchone()
            grounding_result = dict(grounding_result) if grounding_result else {}
            
            # Grounding detections
            if grounding_result:
                cursor.execute("SELECT * FROM grounding_detections WHERE grounding_result_id = ?", 
                             (grounding_result.get('id'),))
                grounding_detections = [dict(row) for row in cursor.fetchall()]
            else:
                grounding_detections = []
            
            conn.close()
            
            return {
                "deteccao": deteccao,
                "florence_captions": captions,
                "florence_objects": objects,
                "grounding_result": grounding_result,
                "grounding_detections": grounding_detections
            }
            
        except Exception as e:
            return {"error": str(e)}
    
    def get_empresa_dados(self, empresa_id):
        """Obtém todos os dados de uma empresa com análises avançadas"""
        db_path = self.base_path / empresa_id / f"{empresa_id}_detection.db"
        
        if not db_path.exists():
            return {"error": "Banco de dados não encontrado"}
        
        try:
            conn = sqlite3.connect(str(db_path))
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            
            # Buscar detecções com joins completos
            deteccoes_query = """
            SELECT d.*, g.max_confidence, g.oil_spill_detected, g.total_detections,
                   g.image_width, g.image_height
            FROM deteccoes d
            LEFT JOIN grounding_dino_results g ON d.id = g.deteccao_id
            ORDER BY d.timestamp_coleta DESC
            """
            
            cursor.execute(deteccoes_query)
            deteccoes = [dict(row) for row in cursor.fetchall()]
            
            # Buscar dados da empresa
            cursor.execute("SELECT * FROM empresa LIMIT 1")
            empresa_row = cursor.fetchone()
            empresa = dict(empresa_row) if empresa_row else {}
            
            # Buscar coletores
            cursor.execute("SELECT * FROM coletores")
            coletores = [dict(row) for row in cursor.fetchall()]
            
            # Análises avançadas
            stats = self.calculate_advanced_stats(deteccoes, cursor)
            
            # Dados para gráficos temporais
            timeline_data = self.get_timeline_data(deteccoes)
            
            # Análise de confiança
            confidence_analysis = self.analyze_confidence_distribution(deteccoes)
            
            # Análise por coletor
            collector_analysis = self.analyze_by_collector(deteccoes)
            
            conn.close()
            
            return {
                "empresa": empresa,
                "coletores": coletores,
                "deteccoes": deteccoes,
                "stats": stats,
                "timeline_data": timeline_data,
                "confidence_analysis": confidence_analysis,
                "collector_analysis": collector_analysis
            }
            
        except Exception as e:
            return {"error": str(e)}
    
    def calculate_advanced_stats(self, deteccoes, cursor):
        """Calcula estatísticas avançadas"""
        total_deteccoes = len(deteccoes)
        deteccoes_positivas = sum(1 for d in deteccoes if d.get('oil_spill_detected', 0) == 1)
        
        # Análise de confiança
        confidences = [d.get('max_confidence', 0) for d in deteccoes if d.get('max_confidence')]
        avg_confidence = sum(confidences) / len(confidences) if confidences else 0
        
        # Análise YOLO vs Grounding DINO
        yolo_detections = sum(d.get('objects_detected_yolo', 0) for d in deteccoes)
        grounding_detections = sum(d.get('total_detections', 0) for d in deteccoes)
        
        # Análise temporal (últimas 24h, 7 dias, 30 dias)
        from datetime import datetime, timedelta
        now = datetime.now()
        
        recent_24h = sum(1 for d in deteccoes 
                        if self.parse_timestamp(d.get('timestamp_coleta', '')) > now - timedelta(hours=24))
        recent_7d = sum(1 for d in deteccoes 
                       if self.parse_timestamp(d.get('timestamp_coleta', '')) > now - timedelta(days=7))
        
        # Contagem de captions Florence
        cursor.execute("SELECT COUNT(*) as total FROM florence_captions")
        total_captions = cursor.fetchone()[0]
        
        # Contagem de objetos Florence
        cursor.execute("SELECT COUNT(*) as total FROM florence_objects")
        total_objects = cursor.fetchone()[0]
        
        return {
            "total_deteccoes": total_deteccoes,
            "deteccoes_positivas": deteccoes_positivas,
            "taxa_deteccao": round((deteccoes_positivas / total_deteccoes * 100) if total_deteccoes > 0 else 0, 2),
            "confianca_media": round(avg_confidence * 100, 2),
            "yolo_detections": yolo_detections,
            "grounding_detections": grounding_detections,
            "recent_24h": recent_24h,
            "recent_7d": recent_7d,
            "total_captions": total_captions,
            "total_objects": total_objects,
            "confidence_ranges": {
                "high": sum(1 for c in confidences if c >= 0.7),
                "medium": sum(1 for c in confidences if 0.5 <= c < 0.7),
                "low": sum(1 for c in confidences if c < 0.5)
            }
        }
    
    def parse_timestamp(self, timestamp_str):
        """Parse timestamp string to datetime"""
        try:
            return datetime.fromisoformat(timestamp_str.replace('T', ' ').split('.')[0])
        except:
            return datetime.min
    
    def get_timeline_data(self, deteccoes):
        """Prepara dados para gráfico temporal"""
        from collections import defaultdict
        
        daily_counts = defaultdict(lambda: {"total": 0, "positivas": 0})
        
        for deteccao in deteccoes:
            try:
                dt = self.parse_timestamp(deteccao.get('timestamp_coleta', ''))
                date_key = dt.strftime('%Y-%m-%d')
                daily_counts[date_key]["total"] += 1
                if deteccao.get('oil_spill_detected', 0) == 1:
                    daily_counts[date_key]["positivas"] += 1
            except:
                continue
        
        return dict(daily_counts)
    
    def analyze_confidence_distribution(self, deteccoes):
        """Analisa distribuição de confiança"""
        confidences = [d.get('max_confidence', 0) for d in deteccoes if d.get('max_confidence')]
        
        if not confidences:
            return {"ranges": {}, "average": 0, "max": 0, "min": 0}
        
        ranges = {
            "0-20%": sum(1 for c in confidences if 0 <= c < 0.2),
            "20-40%": sum(1 for c in confidences if 0.2 <= c < 0.4),
            "40-60%": sum(1 for c in confidences if 0.4 <= c < 0.6),
            "60-80%": sum(1 for c in confidences if 0.6 <= c < 0.8),
            "80-100%": sum(1 for c in confidences if 0.8 <= c <= 1.0)
        }
        
        return {
            "ranges": ranges,
            "average": round(sum(confidences) / len(confidences) * 100, 2),
            "max": round(max(confidences) * 100, 2),
            "min": round(min(confidences) * 100, 2)
        }
    
    def analyze_by_collector(self, deteccoes):
        """Análise por coletor"""
        from collections import defaultdict
        
        collector_stats = defaultdict(lambda: {"total": 0, "positivas": 0, "avg_confidence": 0, "confidences": []})
        
        for deteccao in deteccoes:
            coletor = deteccao.get('coletor_id', 'Unknown')
            collector_stats[coletor]["total"] += 1
            
            if deteccao.get('oil_spill_detected', 0) == 1:
                collector_stats[coletor]["positivas"] += 1
            
            if deteccao.get('max_confidence'):
                collector_stats[coletor]["confidences"].append(deteccao['max_confidence'])
        
        # Calcular médias
        for coletor, stats in collector_stats.items():
            if stats["confidences"]:
                stats["avg_confidence"] = round(sum(stats["confidences"]) / len(stats["confidences"]) * 100, 2)
            stats["taxa_deteccao"] = round((stats["positivas"] / stats["total"] * 100) if stats["total"] > 0 else 0, 2)
            del stats["confidences"]  # Remove lista para JSON
        
        return dict(collector_stats)

    def get_dashboard_template(self):
        """Retorna o template HTML melhorado da dashboard"""
        html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Dashboard - Detecção de Óleo</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/3.9.1/chart.min.js"></script>
    <style>
        * { box-sizing: border-box; }
        body { 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; 
            margin: 0; 
            padding: 20px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
        }
        .container { max-width: 1400px; margin: 0 auto; }
        .header { 
            background: rgba(255,255,255,0.95); 
            color: #2c3e50; 
            padding: 30px; 
            border-radius: 15px; 
            margin-bottom: 30px; 
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            backdrop-filter: blur(10px);
        }
        .header h1 { margin: 0; font-size: 2.5em; text-align: center; }
        .header p { margin: 10px 0 0 0; text-align: center; color: #666; font-size: 1.1em; }
        
        .grid { display: grid; gap: 20px; margin-bottom: 30px; }
        .grid-2 { grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); }
        .grid-3 { grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); }
        .grid-4 { grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); }
        
        .card { 
            background: rgba(255,255,255,0.95); 
            padding: 25px; 
            border-radius: 15px; 
            box-shadow: 0 8px 32px rgba(0,0,0,0.1); 
            backdrop-filter: blur(10px);
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }
        .card:hover { 
            transform: translateY(-5px); 
            box-shadow: 0 12px 40px rgba(0,0,0,0.15); 
        }
        
        .card h3 { margin: 0 0 20px 0; color: #2c3e50; font-size: 1.3em; }
        
        .stats-grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(120px, 1fr)); gap: 15px; }
        .stat { text-align: center; padding: 15px; background: #f8f9fa; border-radius: 10px; }
        .stat-number { 
            font-size: 2.2em; 
            font-weight: bold; 
            margin-bottom: 5px;
            background: linear-gradient(45deg, #3498db, #2ecc71);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
        }
        .stat-label { color: #666; font-size: 0.9em; font-weight: 500; }
        .stat.positive .stat-number { 
            background: linear-gradient(45deg, #e74c3c, #c0392b);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
        }
        .stat.success .stat-number { 
            background: linear-gradient(45deg, #27ae60, #2ecc71);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
        }
        
        table { width: 100%; border-collapse: collapse; background: white; border-radius: 10px; overflow: hidden; }
        th, td { padding: 15px; text-align: left; border-bottom: 1px solid #eee; }
        th { background: #34495e; color: white; font-weight: 600; }
        tr:hover { background: #f8f9fa; }
        
        .status-positivo { 
            color: #27ae60; 
            font-weight: bold; 
            padding: 5px 10px; 
            background: #d5f4e6; 
            border-radius: 20px; 
            font-size: 0.85em;
        }
        .status-negativo { 
            color: #e74c3c; 
            padding: 5px 10px; 
            background: #fadbd8; 
            border-radius: 20px; 
            font-size: 0.85em;
        }
        
        .confidence { font-weight: bold; padding: 5px 10px; border-radius: 20px; font-size: 0.85em; }
        .confidence.high { color: #27ae60; background: #d5f4e6; }
        .confidence.medium { color: #f39c12; background: #fef9e7; }
        .confidence.low { color: #e74c3c; background: #fadbd8; }
        
        select { 
            padding: 12px 15px; 
            margin-right: 15px; 
            border: 2px solid #ddd; 
            border-radius: 8px; 
            font-size: 1em;
            background: white;
            min-width: 200px;
        }
        select:focus { outline: none; border-color: #3498db; }
        
        .loading { text-align: center; padding: 40px; color: #666; font-style: italic; }
        
        .chart-container { 
            position: relative; 
            height: 300px; 
            margin-top: 20px; 
        }
        
        .info-item { 
            display: flex; 
            justify-content: space-between; 
            padding: 10px 0; 
            border-bottom: 1px solid #eee; 
        }
        .info-item:last-child { border-bottom: none; }
        .info-label { font-weight: 600; color: #555; }
        .info-value { color: #333; }
        
        .collector-card { 
            background: #f8f9fa; 
            padding: 15px; 
            border-radius: 10px; 
            margin-bottom: 15px; 
            border-left: 4px solid #3498db;
        }
        
        .btn { 
            padding: 8px 15px; 
            background: #3498db; 
            color: white; 
            border: none; 
            border-radius: 5px; 
            cursor: pointer; 
            font-size: 0.9em;
            transition: background 0.3s ease;
        }
        .btn:hover { background: #2980b9; }
        
        .modal { 
            display: none; 
            position: fixed; 
            z-index: 1000; 
            left: 0; 
            top: 0; 
            width: 100%; 
            height: 100%; 
            background: rgba(0,0,0,0.8); 
        }
        .modal-content { 
            background: white; 
            margin: 5% auto; 
            padding: 30px; 
            width: 80%; 
            max-width: 800px; 
            border-radius: 15px; 
            max-height: 80vh; 
            overflow-y: auto; 
        }
        .close { 
            color: #aaa; 
            float: right; 
            font-size: 28px; 
            font-weight: bold; 
            cursor: pointer; 
        }
        .close:hover { color: black; }
        
        .timeline-chart { height: 250px; }
        .confidence-chart { height: 200px; }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🛢️ Dashboard - Sistema de Detecção de Óleo</h1>
            <p>Monitoramento inteligente com IA para detecção de vazamentos de petróleo</p>
        </div>
        
        <div class="card">
            <h3>🏢 Selecionar Empresa</h3>
            <select id="empresaSelect" onchange="carregarDados()">
                <option value="">Carregando empresas...</option>
            </select>
        </div>
        
        <div id="dashboardContent" style="display: none;">
            <!-- Estatísticas Principais -->
            <div class="card">
                <h3>📊 Estatísticas Gerais</h3>
                <div class="stats-grid">
                    <div class="stat">
                        <div class="stat-number" id="totalDeteccoes">0</div>
                        <div class="stat-label">Total Detecções</div>
                    </div>
                    <div class="stat positive">
                        <div class="stat-number" id="deteccoesPositivas">0</div>
                        <div class="stat-label">Detecções Positivas</div>
                    </div>
                    <div class="stat success">
                        <div class="stat-number" id="taxaDeteccao">0%</div>
                        <div class="stat-label">Taxa de Detecção</div>
                    </div>
                    <div class="stat">
                        <div class="stat-number" id="confiancaMedia">0%</div>
                        <div class="stat-label">Confiança Média</div>
                    </div>
                    <div class="stat">
                        <div class="stat-number" id="recent24h">0</div>
                        <div class="stat-label">Últimas 24h</div>
                    </div>
                    <div class="stat">
                        <div class="stat-number" id="recent7d">0</div>
                        <div class="stat-label">Últimos 7 dias</div>
                    </div>
                </div>
            </div>
            
            <!-- Gráficos -->
            <div class="grid grid-2">
                <div class="card">
                    <h3>📈 Timeline de Detecções</h3>
                    <div class="chart-container timeline-chart">
                        <canvas id="timelineChart"></canvas>
                    </div>
                </div>
                <div class="card">
                    <h3>🎯 Distribuição de Confiança</h3>
                    <div class="chart-container confidence-chart">
                        <canvas id="confidenceChart"></canvas>
                    </div>
                </div>
            </div>
            
            <!-- Informações da Empresa e Coletores -->
            <div class="grid grid-2">
                <div class="card">
                    <h3>🏢 Informações da Empresa</h3>
                    <div id="empresaInfo">Carregando...</div>
                </div>
                <div class="card">
                    <h3>📡 Coletores Ativos</h3>
                    <div id="coletoresInfo">Carregando...</div>
                </div>
            </div>
            
            <!-- Análise por Coletor -->
            <div class="card">
                <h3>📊 Análise por Coletor</h3>
                <div id="collectorAnalysis">Carregando...</div>
            </div>
            
            <!-- Tabela de Detecções -->
            <div class="card">
                <h3>🔍 Detecções Recentes</h3>
                <div style="overflow-x: auto;">
                    <table id="deteccoesTable">
                        <thead>
                            <tr>
                                <th>ID</th>
                                <th>Coletor</th>
                                <th>Data/Hora</th>
                                <th>Modelo</th>
                                <th>Confiança YOLO</th>
                                <th>Confiança Grounding</th>
                                <th>Status</th>
                                <th>Objetos YOLO</th>
                                <th>Det. Grounding</th>
                                <th>Ações</th>
                            </tr>
                        </thead>
                        <tbody>
                            <tr><td colspan="10" class="loading">Selecione uma empresa para ver as detecções</td></tr>
                        </tbody>
                    </table>
                </div>
            </div>
        </div>
    </div>

    <!-- Modal para detalhes -->
    <div id="detailModal" class="modal">
        <div class="modal-content">
            <span class="close">&times;</span>
            <h2>Detalhes da Detecção</h2>
            <div id="modalContent">Carregando...</div>
        </div>
    </div>

    <script>
        let timelineChart, confidenceChart;

        // Carregar lista de empresas
        fetch('/api/empresas')
        .then(response => response.json())
        .then(empresas => {
            const select = document.getElementById('empresaSelect');
            select.innerHTML = '<option value="">Selecione uma empresa</option>';
            empresas.forEach(empresa => {
                select.innerHTML += `<option value="${empresa.id}">${empresa.id}</option>`;
            });
        });

        function carregarDados() {
            const empresaId = document.getElementById('empresaSelect').value;
            if (!empresaId) {
                document.getElementById('dashboardContent').style.display = 'none';
                return;
            }

            document.getElementById('dashboardContent').style.display = 'block';
            
            fetch(`/api/empresa/${empresaId}/dados`)
            .then(response => response.json())
            .then(data => {
                if (data.error) {
                    alert('Erro: ' + data.error);
                    return;
                }
                
                atualizarEstatisticas(data.stats);
                atualizarInformacoes(data.empresa, data.coletores);
                atualizarAnaliseColetores(data.collector_analysis);
                atualizarTabela(data.deteccoes);
                criarGraficos(data.timeline_data, data.confidence_analysis);
                
                // Armazenar empresa_id para uso posterior
                window.currentEmpresaId = empresaId;
            })
            .catch(error => {
                console.error('Erro:', error);
                alert('Erro ao carregar dados');
            });
        }

        function atualizarEstatisticas(stats) {
            document.getElementById('totalDeteccoes').textContent = stats.total_deteccoes;
            document.getElementById('deteccoesPositivas').textContent = stats.deteccoes_positivas;
            document.getElementById('taxaDeteccao').textContent = stats.taxa_deteccao + '%';
            document.getElementById('confiancaMedia').textContent = stats.confianca_media + '%';
            document.getElementById('recent24h').textContent = stats.recent_24h;
            document.getElementById('recent7d').textContent = stats.recent_7d;
        }

        function atualizarInformacoes(empresa, coletores) {
            // Informações da empresa
            document.getElementById('empresaInfo').innerHTML = `
                <div class="info-item">
                    <span class="info-label">ID:</span>
                    <span class="info-value">${empresa.empresa_id || 'N/A'}</span>
                </div>
                <div class="info-item">
                    <span class="info-label">Nome:</span>
                    <span class="info-value">${empresa.empresa_nome || 'N/A'}</span>
                </div>
                <div class="info-item">
                    <span class="info-label">Criado em:</span>
                    <span class="info-value">${new Date(empresa.created_at).toLocaleString('pt-BR') || 'N/A'}</span>
                </div>
                <div class="info-item">
                    <span class="info-label">Última atualização:</span>
                    <span class="info-value">${new Date(empresa.updated_at).toLocaleString('pt-BR') || 'N/A'}</span>
                </div>
            `;
            
            // Coletores
            let coletoresHtml = '';
            coletores.forEach(coletor => {
                coletoresHtml += `
                    <div class="collector-card">
                        <div class="info-item">
                            <span class="info-label">ID:</span>
                            <span class="info-value">${coletor.coletor_id}</span>
                        </div>
                        <div class="info-item">
                            <span class="info-label">Descrição:</span>
                            <span class="info-value">${coletor.coletor_descricao || 'Sem descrição'}</span>
                        </div>
                        <div class="info-item">
                            <span class="info-label">Localização:</span>
                            <span class="info-value">${coletor.coletor_localizacao || 'N/A'}</span>
                        </div>
                        <div class="info-item">
                            <span class="info-label">Status:</span>
                            <span class="info-value" style="color: ${coletor.status === 'ativo' ? '#27ae60' : '#e74c3c'}">${coletor.status || 'N/A'}</span>
                        </div>
                    </div>
                `;
            });
            document.getElementById('coletoresInfo').innerHTML = coletoresHtml || 'Nenhum coletor encontrado';
        }

        function atualizarAnaliseColetores(collectorAnalysis) {
            let analysisHtml = '';
            Object.entries(collectorAnalysis).forEach(([coletor, stats]) => {
                analysisHtml += `
                    <div class="collector-card">
                        <h4 style="margin: 0 0 15px 0; color: #2c3e50;">${coletor}</h4>
                        <div class="stats-grid" style="grid-template-columns: repeat(4, 1fr);">
                            <div class="stat">
                                <div class="stat-number" style="font-size: 1.5em;">${stats.total}</div>
                                <div class="stat-label">Total</div>
                            </div>
                            <div class="stat positive">
                                <div class="stat-number" style="font-size: 1.5em;">${stats.positivas}</div>
                                <div class="stat-label">Positivas</div>
                            </div>
                            <div class="stat success">
                                <div class="stat-number" style="font-size: 1.5em;">${stats.taxa_deteccao}%</div>
                                <div class="stat-label">Taxa</div>
                            </div>
                            <div class="stat">
                                <div class="stat-number" style="font-size: 1.5em;">${stats.avg_confidence}%</div>
                                <div class="stat-label">Confiança Média</div>
                            </div>
                        </div>
                    </div>
                `;
            });
            document.getElementById('collectorAnalysis').innerHTML = analysisHtml || 'Nenhuma análise disponível';
        }

        function atualizarTabela(deteccoes) {
            const tbody = document.querySelector('#deteccoesTable tbody');
            tbody.innerHTML = '';
            
            if (deteccoes.length === 0) {
                tbody.innerHTML = '<tr><td colspan="10" class="loading">Nenhuma detecção encontrada</td></tr>';
            } else {
                deteccoes.forEach(deteccao => {
                    const yoloConfidence = deteccao.confidence_yolo || 0;
                    const groundingConfidence = deteccao.max_confidence || 0;
                    
                    const yoloClass = yoloConfidence >= 0.8 ? 'high' : yoloConfidence >= 0.6 ? 'medium' : 'low';
                    const groundingClass = groundingConfidence >= 0.8 ? 'high' : groundingConfidence >= 0.6 ? 'medium' : 'low';
                    
                    const status = deteccao.oil_spill_detected ? 'Óleo Detectado' : 'Sem Óleo';
                    const statusClass = deteccao.oil_spill_detected ? 'status-positivo' : 'status-negativo';
                    
                    tbody.innerHTML += `
                        <tr>
                            <td>${deteccao.id}</td>
                            <td>${deteccao.coletor_id}</td>
                            <td>${new Date(deteccao.timestamp_coleta).toLocaleString('pt-BR')}</td>
                            <td>${deteccao.modelo_usado}</td>
                            <td><span class="confidence ${yoloClass}">${(yoloConfidence * 100).toFixed(1)}%</span></td>
                            <td><span class="confidence ${groundingClass}">${(groundingConfidence * 100).toFixed(1)}%</span></td>
                            <td><span class="${statusClass}">${status}</span></td>
                            <td>${deteccao.objects_detected_yolo || 0}</td>
                            <td>${deteccao.total_detections || 0}</td>
                            <td><button class="btn" onclick="verDetalhes(${deteccao.id})">Ver Detalhes</button></td>
                        </tr>
                    `;
                });
            }
        }

        function criarGraficos(timelineData, confidenceAnalysis) {
            // Destruir gráficos existentes
            if (timelineChart) timelineChart.destroy();
            if (confidenceChart) confidenceChart.destroy();

            // Gráfico de Timeline
            const timelineCtx = document.getElementById('timelineChart').getContext('2d');
            const sortedDates = Object.keys(timelineData).sort();
            
            timelineChart = new Chart(timelineCtx, {
                type: 'line',
                data: {
                    labels: sortedDates.map(date => new Date(date).toLocaleDateString('pt-BR')),
                    datasets: [{
                        label: 'Total de Detecções',
                        data: sortedDates.map(date => timelineData[date].total),
                        borderColor: '#3498db',
                        backgroundColor: 'rgba(52, 152, 219, 0.1)',
                        tension: 0.4,
                        fill: true
                    }, {
                        label: 'Detecções Positivas',
                        data: sortedDates.map(date => timelineData[date].positivas),
                        borderColor: '#e74c3c',
                        backgroundColor: 'rgba(231, 76, 60, 0.1)',
                        tension: 0.4,
                        fill: true
                    }]
                },
                options: {
                    responsive: true,
                    maintainAspectRatio: false,
                    plugins: {
                        legend: {
                            position: 'top'
                        }
                    },
                    scales: {
                        y: {
                            beginAtZero: true,
                            ticks: {
                                stepSize: 1
                            }
                        }
                    }
                }
            });

            // Gráfico de Distribuição de Confiança
            const confidenceCtx = document.getElementById('confidenceChart').getContext('2d');
            
            confidenceChart = new Chart(confidenceCtx, {
                type: 'doughnut',
                data: {
                    labels: Object.keys(confidenceAnalysis.ranges),
                    datasets: [{
                        data: Object.values(confidenceAnalysis.ranges),
                        backgroundColor: [
                            '#e74c3c',
                            '#f39c12',
                            '#f1c40f',
                            '#2ecc71',
                            '#27ae60'
                        ],
                        borderWidth: 2,
                        borderColor: '#fff'
                    }]
                },
                options: {
                    responsive: true,
                    maintainAspectRatio: false,
                    plugins: {
                        legend: {
                            position: 'bottom'
                        }
                    }
                }
            });
        }

        function verDetalhes(deteccaoId) {
            const modal = document.getElementById('detailModal');
            const modalContent = document.getElementById('modalContent');
            
            modal.style.display = 'block';
            modalContent.innerHTML = 'Carregando detalhes...';
            
            fetch(`/api/empresa/${window.currentEmpresaId}/deteccao/${deteccaoId}`)
            .then(response => response.json())
            .then(data => {
                if (data.error) {
                    modalContent.innerHTML = `<p style="color: red;">Erro: ${data.error}</p>`;
                    return;
                }
                
                const deteccao = data.deteccao;
                const captions = data.florence_captions;
                const objects = data.florence_objects;
                const groundingResult = data.grounding_result;
                const groundingDetections = data.grounding_detections;
                
                let detalhesHtml = `
                    <div class="grid grid-2" style="gap: 20px;">
                        <div>
                            <h3>📋 Informações Básicas</h3>
                            <div class="info-item">
                                <span class="info-label">ID:</span>
                                <span class="info-value">${deteccao.id}</span>
                            </div>
                            <div class="info-item">
                                <span class="info-label">Coletor:</span>
                                <span class="info-value">${deteccao.coletor_id}</span>
                            </div>
                            <div class="info-item">
                                <span class="info-label">Coleta:</span>
                                <span class="info-value">${new Date(deteccao.timestamp_coleta).toLocaleString('pt-BR')}</span>
                            </div>
                            <div class="info-item">
                                <span class="info-label">Processamento:</span>
                                <span class="info-value">${new Date(deteccao.timestamp_processamento).toLocaleString('pt-BR')}</span>
                            </div>
                            <div class="info-item">
                                <span class="info-label">Modelo:</span>
                                <span class="info-value">${deteccao.modelo_usado}</span>
                            </div>
                        </div>
                        
                        <div>
                            <h3>🎯 Resultados YOLO</h3>
                            <div class="info-item">
                                <span class="info-label">Confiança:</span>
                                <span class="info-value">${(deteccao.confidence_yolo * 100).toFixed(2)}%</span>
                            </div>
                            <div class="info-item">
                                <span class="info-label">Objetos Detectados:</span>
                                <span class="info-value">${deteccao.objects_detected_yolo}</span>
                            </div>
                        </div>
                    </div>
                `;
                
                if (groundingResult && Object.keys(groundingResult).length > 0) {
                    detalhesHtml += `
                        <div style="margin-top: 20px;">
                            <h3>🔍 Resultados Grounding DINO</h3>
                            <div class="grid grid-3">
                                <div class="info-item">
                                    <span class="info-label">Óleo Detectado:</span>
                                    <span class="info-value" style="color: ${groundingResult.oil_spill_detected ? '#27ae60' : '#e74c3c'}">
                                        ${groundingResult.oil_spill_detected ? 'Sim' : 'Não'}
                                    </span>
                                </div>
                                <div class="info-item">
                                    <span class="info-label">Confiança Máxima:</span>
                                    <span class="info-value">${(groundingResult.max_confidence * 100).toFixed(2)}%</span>
                                </div>
                                <div class="info-item">
                                    <span class="info-label">Total Detecções:</span>
                                    <span class="info-value">${groundingResult.total_detections}</span>
                                </div>
                                <div class="info-item">
                                    <span class="info-label">Largura da Imagem:</span>
                                    <span class="info-value">${groundingResult.image_width}px</span>
                                </div>
                                <div class="info-item">
                                    <span class="info-label">Altura da Imagem:</span>
                                    <span class="info-value">${groundingResult.image_height}px</span>
                                </div>
                            </div>
                        </div>
                    `;
                }
                
                if (groundingDetections && groundingDetections.length > 0) {
                    detalhesHtml += `
                        <div style="margin-top: 20px;">
                            <h3>📍 Detecções Específicas</h3>
                            <div style="max-height: 200px; overflow-y: auto;">
                                <table style="font-size: 0.9em;">
                                    <thead>
                                        <tr>
                                            <th>Frase</th>
                                            <th>Confiança</th>
                                            <th>Bounding Box (x1, y1, x2, y2)</th>
                                        </tr>
                                    </thead>
                                    <tbody>
                `;
                
                groundingDetections.forEach(det => {
                    const confidence = (det.confidence * 100).toFixed(2);
                    const bbox = `(${det.bbox_x1.toFixed(3)}, ${det.bbox_y1.toFixed(3)}, ${det.bbox_x2.toFixed(3)}, ${det.bbox_y2.toFixed(3)})`;
                    detalhesHtml += `
                        <tr>
                            <td>${det.phrase.replace(' [SEP]', '')}</td>
                            <td><span class="confidence ${det.confidence >= 0.7 ? 'high' : det.confidence >= 0.5 ? 'medium' : 'low'}">${confidence}%</span></td>
                            <td style="font-family: monospace; font-size: 0.8em;">${bbox}</td>
                        </tr>
                    `;
                });
                
                detalhesHtml += `
                                    </tbody>
                                </table>
                            </div>
                        </div>
                    `;
                }
                
                if (captions && captions.length > 0) {
                    detalhesHtml += `
                        <div style="margin-top: 20px;">
                            <h3>💬 Descrições Florence</h3>
                    `;
                    
                    captions.forEach(caption => {
                        detalhesHtml += `
                            <div class="collector-card">
                                <div class="info-item">
                                    <span class="info-label">Tipo:</span>
                                    <span class="info-value">${caption.caption_type}</span>
                                </div>
                                <div class="info-item">
                                    <span class="info-label">Texto:</span>
                                    <span class="info-value">${caption.caption_text}</span>
                                </div>
                            </div>
                        `;
                    });
                    
                    detalhesHtml += `</div>`;
                }
                
                if (objects && objects.length > 0) {
                    detalhesHtml += `
                        <div style="margin-top: 20px;">
                            <h3>🎯 Objetos Detectados (Florence)</h3>
                            <div style="max-height: 200px; overflow-y: auto;">
                                <table style="font-size: 0.9em;">
                                    <thead>
                                        <tr>
                                            <th>Objeto</th>
                                            <th>Bounding Box (x1, y1, x2, y2)</th>
                                        </tr>
                                    </thead>
                                    <tbody>
                    `;
                    
                    objects.forEach(obj => {
                        const bbox = `(${obj.bbox_x1.toFixed(1)}, ${obj.bbox_y1.toFixed(1)}, ${obj.bbox_x2.toFixed(1)}, ${obj.bbox_y2.toFixed(1)})`;
                        detalhesHtml += `
                            <tr>
                                <td><strong>${obj.object_label}</strong></td>
                                <td style="font-family: monospace; font-size: 0.8em;">${bbox}</td>
                            </tr>
                        `;
                    });
                    
                    detalhesHtml += `
                                    </tbody>
                                </table>
                            </div>
                        </div>
                    `;
                }
                
                detalhesHtml += `
                    <div style="margin-top: 20px;">
                        <h3>📁 Caminhos dos Arquivos</h3>
                        <div class="info-item">
                            <span class="info-label">Imagem Original:</span>
                            <span class="info-value" style="font-family: monospace; font-size: 0.8em; word-break: break-all;">${deteccao.imagem_original_path}</span>
                        </div>
                        <div class="info-item">
                            <span class="info-label">Imagem Processada:</span>
                            <span class="info-value" style="font-family: monospace; font-size: 0.8em; word-break: break-all;">${deteccao.imagem_processada_path}</span>
                        </div>
                    </div>
                `;
                
                modalContent.innerHTML = detalhesHtml;
            })
            .catch(error => {
                console.error('Erro:', error);
                modalContent.innerHTML = `<p style="color: red;">Erro ao carregar detalhes: ${error.message}</p>`;
            });
        }

        // Configurar modal
        const modal = document.getElementById('detailModal');
        const closeBtn = document.getElementsByClassName('close')[0];
        
        closeBtn.onclick = function() {
            modal.style.display = 'none';
        }
        
        window.onclick = function(event) {
            if (event.target == modal) {
                modal.style.display = 'none';
            }
        }
    </script>
</body>
</html>
        """
        return html_template

In [38]:
class OilDetectionServer:
    """Servidor principal para detecção de óleo"""
    
    def __init__(self):
        self.app = Flask(__name__)
        self.db_manager = DatabaseManager()
        self.file_manager = FileManager()
        self.model_manager = ModelManager()
        self.web_dashboard = WebDashboard(self.db_manager, self.file_manager)
        self._setup_routes()
    def _setup_routes(self):
        """Configura as rotas do Flask"""
        self.web_dashboard.setup_routes(self.app)
        
        @self.app.route('/oil_detection', methods=['POST'])
        def oil_detection_endpoint():
            return self.process_oil_detection()
        
        @self.app.route('/processar', methods=['POST'])
        def processar():
            return self.legacy_processar()
        
        @self.app.route('/skf_analysis', methods=['POST'])
        def handle_skf_analysis():
            return self.skf_analysis_endpoint()
        
        @self.app.route('/health', methods=['GET'])
        def health_check():
            return jsonify({"status": "healthy", "timestamp": datetime.now().isoformat()})
        
        @self.app.route('/stats', methods=['GET'])
        def get_stats():
            return self.get_detection_stats()
        
        @self.app.route('/stats/<empresa_id>', methods=['GET'])
        def get_empresa_stats(empresa_id):
            return self.get_empresa_stats(empresa_id)
        
        @self.app.route('/detection/<empresa_id>/<int:deteccao_id>', methods=['GET'])
        def get_detection_details(empresa_id, deteccao_id):
            return self.get_detection_details(empresa_id, deteccao_id)
            
        @self.app.route('/', methods=['GET'])
        def dashboard():
            return self.web_dashboard.get_dashboard_template()

        @self.app.route('/empresa/<empresa_id>/detections', methods=['GET'])
        def get_empresa_detections(empresa_id):
            limit = request.args.get('limit', 10, type=int)
            detections = self.web_dashboard.get_empresa_detections_list(empresa_id, limit)
            
            if "error" in detections:
                return jsonify({
                    "status": "error",
                    "message": detections["error"]
                }), 404
            
            return jsonify({
                "status": "success",
                "data": detections
            })

        @self.app.route('/image/<path:image_path>')
        def serve_image(image_path):
            """Serve imagens do sistema de arquivos"""
            try:
                full_path = Path(self.file_manager.base_path) / image_path
                
                if full_path.exists() and full_path.is_file():
                    return send_file(str(full_path), mimetype='image/jpeg')
                else:
                    return jsonify({"error": "Image not found"}), 404
                    
            except Exception as e:
                logger.error(f"Erro ao servir imagem {image_path}: {e}")
                return jsonify({"error": "Error serving image"}), 500
    
    def process_oil_detection(self):
        """Endpoint principal para processamento de detecção de óleo"""
        try:
            data = request.get_json()
            
            # Validar dados obrigatórios
            required_fields = ['imagem_original_base64', 'empresa_info']
            for field in required_fields:
                if field not in data:
                    return jsonify({
                        "status": "error", 
                        "message": f"Campo obrigatório ausente: {field}"
                    }), 400
            
            # Extrair dados da estrutura empresa_info
            empresa_info = data['empresa_info']
            empresa_id = empresa_info['empresa_id']
            empresa_nome = empresa_info['empresa_nome']
            coletor_id = empresa_info['coletor_id']
            coletor_descricao = empresa_info.get('coletor_descricao', '')
            coletor_localizacao = empresa_info.get('localizacao', '')
            modelo_usado = empresa_info.get('modelo_id', 'oil_spill')
            confidence_yolo = data.get('confidence_threshold')
            timestamp_coleta = data.get('timestamp', datetime.now().isoformat())
            
            # Validação e conversão segura da imagem
            try:
                imagem_bytes = base64.b64decode(data['imagem_original_base64'])
                image = Image.open(io.BytesIO(imagem_bytes)).convert("RGB")
                logger.info(f"Image loaded successfully: {image.size}")
            except Exception as e:
                logger.error(f"Erro ao processar imagem base64: {e}")
                return jsonify({
                    "status": "error", 
                    "message": f"Erro ao processar imagem: {str(e)}"
                }), 400
            
            # Inserir/atualizar empresa e coletor no banco específico da empresa
            self.db_manager.insert_empresa(empresa_id, empresa_nome)
            self.db_manager.insert_coletor(empresa_id, coletor_id, coletor_descricao, coletor_localizacao)
            
            # Criar estrutura de diretórios específica da empresa
            raw_dir, processed_dir = self.file_manager.create_directory_structure(
                empresa_id, coletor_id, modelo_usado
            )
            
            # Processar com Florence-2
            logger.info(f"Processing image with Florence-2 for {empresa_id}/{coletor_id}")
            florence_results = self.model_manager.generate_florence_captions(image)
            
            # Processar com GroundingDINO
            logger.info(f"Processing image with GroundingDINO for {empresa_id}/{coletor_id}")
            grounding_results = self.model_manager.process_grounding_detection(image)
            
            # Calcular detecções de forma segura
            detection_data = grounding_results.get('detection_data', {})
            logits = detection_data.get('logits', np.array([]))
            
            # Usar contagem de detecções do payload se existir, senão calcular
            if isinstance(logits, np.ndarray):
                objects_detected = data.get('detections_count', int(logits.size))
            else:
                objects_detected = data.get('detections_count', len(logits) if logits else 0)
            
            # Gerar nomes de arquivo
            timestamp_str = datetime.now().strftime("%Y%m%d%H%M%S%f")[:-3]
            raw_filename = data.get('name_imagem_original', 
                                  self.file_manager.generate_filename(
                                      empresa_id, coletor_id, timestamp_str, modelo_usado, objects_detected, "RAW"
                                  ))
            processed_filename = data.get('name_imagem_processada',
                                        self.file_manager.generate_filename(
                                            empresa_id, coletor_id, timestamp_str, modelo_usado, objects_detected, "PROC"
                                        ))
            
            # Salvar imagens
            raw_path = raw_dir / raw_filename
            processed_path = processed_dir / processed_filename
            
            # Verificar se salvou corretamente
            if not self.file_manager.save_image(image, raw_path):
                logger.warning(f"Failed to save raw image to {raw_path}")
            
            # Salvar imagem processada se existir no payload
            processed_image_saved = False
            if 'imagem_processada_base64' in data:
                try:
                    processed_bytes = base64.b64decode(data['imagem_processada_base64'])
                    processed_image = Image.open(io.BytesIO(processed_bytes)).convert("RGB")
                    processed_image_saved = self.file_manager.save_image(processed_image, processed_path)
                except Exception as e:
                    logger.error(f"Erro ao processar imagem processada: {e}")
            elif 'annotated_image' in grounding_results:
                try:
                    annotated_array = grounding_results['annotated_image']
                    if isinstance(annotated_array, np.ndarray) and annotated_array.size > 0:
                        processed_image = Image.fromarray(annotated_array.astype('uint8'))
                        processed_image_saved = self.file_manager.save_image(processed_image, processed_path)
                except Exception as e:
                    logger.error(f"Erro ao salvar imagem anotada: {e}")
            
            # Preparar dados para inserção no banco
            dados_deteccao = {
                'coletor_id': coletor_id,
                'timestamp_coleta': timestamp_coleta,
                'modelo_usado': modelo_usado,
                'imagem_original_path': str(raw_path),
                'imagem_processada_path': str(processed_path) if processed_image_saved else None,
                'confidence_yolo': confidence_yolo,
                'objects_detected_yolo': objects_detected
            }
            
            # Inserir no banco de dados específico da empresa
            deteccao_id = self.db_manager.insert_deteccao(empresa_id, dados_deteccao)
            self.db_manager.insert_florence_result(empresa_id, deteccao_id, florence_results)
            self.db_manager.insert_grounding_result(empresa_id, deteccao_id, grounding_results)
            
            # Calcular max_confidence de forma segura
            if isinstance(logits, np.ndarray):
                max_confidence = float(np.max(logits)) if logits.size > 0 else 0.0
            else:
                max_confidence = float(max(logits)) if logits else 0.0
            
            # Preparar resposta
            response = {
                "status": "success",
                "deteccao_id": deteccao_id,
                "empresa_id": empresa_id,
                "objects_detected": objects_detected,
                "oil_spill_detected": objects_detected > 0,
                "max_confidence": max_confidence,
                "florence_caption": florence_results.get('<CAPTION>', ''),
                "files_saved": {
                    "raw_image": str(raw_path),
                    "processed_image": str(processed_path) if processed_image_saved else None
                },
                "timestamp_processamento": datetime.now().isoformat()
            }
            
            logger.info(f"Successfully processed detection {deteccao_id} for {empresa_id}/{coletor_id}")
            return jsonify(response)
            
        except Exception as e:
            logger.error(f"Erro no processamento: {e}", exc_info=True)
            return jsonify({
                "status": "error",
                "message": f"Erro interno do servidor: {str(e)}"
            }), 500
        
    def legacy_processar(self):
        """Endpoint legacy para compatibilidade"""
        try:
            dados = request.get_json()
            imagem_b64 = dados.get('imagem_base64')
            nome_imagem = dados.get('nome_imagem', 'imagem_recebida.jpg')
            texto = dados.get('texto', '')
            
            if not imagem_b64:
                return jsonify({"resposta": "Imagem base64 não fornecida"}), 400
            
            # Salva a imagem temporariamente
            with open(nome_imagem, "wb") as f:
                f.write(base64.b64decode(imagem_b64))
            
            tamanho_kb = os.path.getsize(nome_imagem) / 1024
            resposta = f"{nome_imagem}: {tamanho_kb:.1f}kb | Texto recebido: {texto}"
            
            return jsonify({"resposta": resposta})
        except Exception as e:
            logger.error(f"Erro no endpoint legacy: {e}")
            return jsonify({"resposta": f"Erro: {str(e)}"}), 500
    
    def skf_analysis_endpoint(self):
        """Endpoint para análise SKF (Florence-2)"""
        try:
            data = request.json
            
            imagem_base64 = data.get("imagem_base64")
            if not imagem_base64:
                return jsonify({"status": "error", "message": "Imagem base64 não fornecida"}), 400
            
            # Converter base64 para imagem
            imagem_bytes = base64.b64decode(imagem_base64)
            image_stream = io.BytesIO(imagem_bytes)
            image = Image.open(image_stream).convert("RGB")
            
            # Processar com Florence-2
            result = self.model_manager.generate_florence_captions(image)
            
            return jsonify({
                "status": "success",
                "return": result
            })
            
        except Exception as e:
            logger.error(f"Erro no SKF analysis: {e}")
            return jsonify({
                "status": "error",
                "message": f"Erro no servidor: {str(e)}"
            }), 500
    
    def get_detection_stats(self):
        """Retorna estatísticas globais de todas as empresas"""
        try:
            # Buscar todas as empresas no diretório base
            base_path = Path(self.db_manager.base_path)
            if not base_path.exists():
                return jsonify({
                    "status": "success",
                    "stats": {
                        "total_empresas": 0,
                        "total_deteccoes": 0,
                        "deteccoes_positivas": 0,
                        "taxa_deteccao": 0,
                        "empresas": []
                    }
                })
            
            empresas_stats = []
            total_deteccoes = 0
            total_positivas = 0
            
            for empresa_dir in base_path.iterdir():
                if empresa_dir.is_dir():
                    empresa_id = empresa_dir.name
                    stats = self.db_manager.get_empresa_stats(empresa_id)
                    
                    if "error" not in stats:
                        empresas_stats.append(stats)
                        total_deteccoes += stats["total_deteccoes"]
                        total_positivas += stats["deteccoes_positivas"]
            
            return jsonify({
                "status": "success",
                "stats": {
                    "total_empresas": len(empresas_stats),
                    "total_deteccoes": total_deteccoes,
                    "deteccoes_positivas": total_positivas,
                    "taxa_deteccao": (total_positivas / total_deteccoes * 100) if total_deteccoes > 0 else 0,
                    "empresas": empresas_stats
                }
            })
            
        except Exception as e:
            logger.error(f"Erro ao obter estatísticas globais: {e}")
            return jsonify({
                "status": "error",
                "message": str(e)
            }), 500
    
    def get_empresa_stats(self, empresa_id: str):
        """Retorna estatísticas específicas de uma empresa"""
        try:
            stats = self.db_manager.get_empresa_stats(empresa_id)
            
            if "error" in stats:
                return jsonify({
                    "status": "error",
                    "message": stats["error"]
                }), 404
            
            # Adicionar informações de diretório
            dir_info = self.file_manager.get_empresa_directory_info(empresa_id)
            if "error" not in dir_info:
                stats["directory_info"] = dir_info
            
            return jsonify({
                "status": "success",
                "stats": stats
            })
            
        except Exception as e:
            logger.error(f"Erro ao obter estatísticas da empresa {empresa_id}: {e}")
            return jsonify({
                "status": "error",
                "message": str(e)
            }), 500
    
    def get_detection_details(self, empresa_id: str, deteccao_id: int):
        """Retorna detalhes completos de uma detecção específica"""
        try:
            detection_data = self.db_manager.get_detection_with_details(empresa_id, deteccao_id)
            
            if not detection_data:
                return jsonify({
                    "status": "error",
                    "message": "Detecção não encontrada"
                }), 404
            
            return jsonify({
                "status": "success",
                "data": detection_data
            })
            
        except Exception as e:
            logger.error(f"Erro ao obter detalhes da detecção {deteccao_id} da empresa {empresa_id}: {e}")
            return jsonify({
                "status": "error",
                "message": str(e)
            }), 500
    
    def run(self, host="0.0.0.0", port=8080, debug=False):
        """Executa o servidor"""
        logger.info(f"Starting Oil Detection Server on {host}:{port}")
        self.app.run(host=host, port=port, debug=debug)

In [ ]:
if __name__ == "__main__":
    if os.environ.get('WERKZEUG_RUN_MAIN') != 'true': 
        print("Initializing Oil Detection Server...")
    
    server = OilDetectionServer()
    debug_mode = "--debug" in sys.argv
    server.run(debug=debug_mode)